https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/

## Access to the shell

In [2]:
%ls       ######## listinf files in the directory

 Volume in drive C is Windows

File Not Found


 Volume Serial Number is 4234-2518

 Directory of C:\Users\ANTHONY\Desktop\important ipynb codes\SPARK


 Directory of C:\Users\ANTHONY\Desktop\important ipynb codes\SPARK


 Directory of C:\Users\ANTHONY\Desktop\important ipynb codes\SPARK


 Directory of C:\Users\ANTHONY\Desktop\important ipynb codes\SPARK


 Directory of C:\Users\ANTHONY\Desktop\important ipynb codes\SPARK


 Directory of C:\Users\ANTHONY\Desktop\important ipynb codes\SPARK



In [4]:
%pwd

'C:\\Users\\ANTHONY\\Desktop\\important ipynb codes\\SPARK'

## INTSALLING SPARK

In [ ]:
#################### to run it in colab. you need these codes #######################33
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
################33 this tooooo    ##################3
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

To start working with Spark DataFrames, you first have to create a SparkSession object from your SparkContext. You can think of the SparkContext as your connection to the cluster and the SparkSession as your interface with that connection.

In [7]:
###############To start working with Spark DataFrames, you first have to create a SparkSession object from your SparkContext.
#######You can think of the SparkContext as your connection to the cluster and the SparkSession as your interface with that connection.


In [5]:


import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

sc

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading and preprocessing Chicago's Reported Crime Data


In [ ]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD&bom=true&query=select+*    
    
    ########## to download data from web directly

In [8]:
%ls

 Volume in drive C is Windows
 Volume Serial Number is 4234-2518

 Directory of C:\Users\ANTHONY\Desktop\important ipynb codes\SPARK

11-05-2021  08:26 AM    <DIR>          .
11-05-2021  08:26 AM    <DIR>          ..
11-05-2021  08:24 AM    <DIR>          .ipynb_checkpoints
11-05-2021  08:24 AM           298,886 1.Context , session and data import.ipynb
11-05-2021  08:23 AM            99,151 2. Schemas_Columns_rows_groupby_sort_unique.ipynb
03-04-2021  08:37 PM            23,185 3. Built_in_functions.ipynb
05-04-2021  09:43 AM            84,261 3.Functions&date&joins.ipynb
03-04-2021  09:23 AM             4,046 Download_data.ipynb
03-04-2021  07:33 PM             2,294 FIRST_SAMPLE.ipynb
05-05-2021  09:05 PM    <DIR>          New Folder
19-08-2019  06:56 AM             5,699 police_station.csv
05-04-2021  10:03 AM            22,987 RDD.ipynb
02-04-2021  04:48 PM       171,057,223 reported-crimes.csv
02-04-2021  04:48 PM       171,057,223 rows.csv@accessType=DOWNLOAD
11-05-2021  08:23 A

In [12]:
############  rows.csv@accessType=DOWNLOAD  is the name of downloaded file... not easily accesible. So we rename it

##############           mv is the shell command for renaming it   #############

%mv rows.csv@accessType=DOWNLOAD reported-crimes.csv

UsageError: Line magic function `%mv` not found.


In [10]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('rows.csv@accessType=DOWNLOAD',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))

####################      only filtering till 2018       #######################

print(rc.show(5))       #######3 to show in  a nice format we use show

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

In [13]:
rc.limit(5)          ######  show and limit are same

DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

In [15]:
rc.show(n=5,truncate=True, vertical=True)         ###### we can use truncate to print like this

-RECORD 0------------------------------------
 ID                   | 10224738             
 Case Number          | HY411648             
 Date                 | 2015-09-05 13:30:00  
 Block                | 043XX S WOOD ST      
 IUCR                 | 0486                 
 Primary Type         | BATTERY              
 Description          | DOMESTIC BATTERY ... 
 Location Description | RESIDENCE            
 Arrest               | false                
 Domestic             | true                 
 Beat                 | 0924                 
 District             | 009                  
 Ward                 | 12                   
 Community Area       | 61                   
 FBI Code             | 08B                  
 X Coordinate         | 1165074              
 Y Coordinate         | 1875917              
 Year                 | 2015                 
 Updated On           | 02/10/2018 03:50:... 
 Latitude             | 41.815117282         
 Longitude            | -87.669999

In [27]:
#############################################   use this code to get horizontal way   ####################################3

%%html
<style>
div.output_area pre {
    white-space: pre;}
</style>

In [30]:
rc.show(50)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| fal